Fatemeh Mohammadi - 810199489

# Step Zero: Import Libraries & Set Parameters

## Import Libraries:

In [ ]:
!pip install librosa numpy soundfile matplotlib noisereduce scipy
!pip install hmmlearn colorama collections scipy

In [ ]:
import librosa
import numpy as np
import os
import soundfile as sf
import matplotlib.pyplot as plt
import noisereduce as nr
from scipy.spatial.distance import euclidean
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from hmmlearn import hmm
from sklearn.model_selection import train_test_split
from colorama import Fore, Style
from collections import defaultdict
import scipy.stats


## Set Parameters 

In [ ]:
initial_recordings_folder = 'recordings'
preprocessed_recordings_folder = 'preprocessed_recordings'
mfcc_features_folder = 'mfcc_features'
heatmaps_folder = "heatmaps"

DIGITS = []
SPEAKERS = []

TARGET_SAMPLING_RATE = 16000

N_MFCC = 13
N_FRAME_MFCC = 30

NUM_REPEATED_RECORDING = 50

TRAIN_PERCENT = 0.3
TEST_PERCENT = 1 - TRAIN_PERCENT

NUM_STATE = 13
NUM_ITERATION = 10

# Step One: Preprocessing & Feature Extraction

## Preprocessing:

In [ ]:
def preprocess_audio(input_path, output_path, target_sampling_rate = TARGET_SAMPLING_RATE):
    # Load the audio file
    audio, sampling_rate = librosa.load(input_path, sr = target_sampling_rate)

    # Noise reduction
    reduced_noise_audio = nr.reduce_noise(y=audio, sr = sampling_rate)

    # Silence removal
    non_silent_audio, _ = librosa.effects.trim(audio)

    # Normalize the audio to a standard volume level
    normalized_audio = librosa.util.normalize(non_silent_audio)

    # Save the processed audio file
    sf.write(output_path, normalized_audio, target_sampling_rate)

In [ ]:
def preprocess_all_audio(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for file_name in os.listdir(input_folder):
        if file_name.endswith('.wav'):
            file_path = os.path.join(input_folder, file_name)
            output_path = os.path.join(output_folder, file_name)
            preprocess_audio(file_path, output_path)
            #print(f"Processed {file_name}")

In [ ]:
preprocess_all_audio(initial_recordings_folder, preprocessed_recordings_folder)

## Feature Extraction

### Extract MFCC

In [ ]:
def pad_mfcc(raw_mfcc, n_frame_mfcc = N_FRAME_MFCC):
    temp = np.tile(raw_mfcc, (1 ,int(np.ceil(n_frame_mfcc / raw_mfcc.shape[1])) ))
    padded_mfcc_features= temp[:,:n_frame_mfcc]
    return padded_mfcc_features

In [ ]:
def extract_mfcc(file_path, target_sampling_rate = TARGET_SAMPLING_RATE, n_mfcc = N_MFCC, n_frame_mfcc = N_FRAME_MFCC):
    audio, _ = librosa.load(file_path)
    raw_mfcc = librosa.feature.mfcc(y = audio, sr = target_sampling_rate, n_mfcc = n_mfcc)
    mfcc = pad_mfcc(raw_mfcc, n_frame_mfcc)
    return mfcc

In [ ]:
def extract_features_for_all_files(recordings_folder, mfcc_folder):
    if not os.path.exists(mfcc_folder):
        os.makedirs(mfcc_folder)

    for file_name in os.listdir(recordings_folder):
        if file_name.endswith('.wav'):
            file_path = os.path.join(recordings_folder, file_name)
            mfcc_feature = extract_mfcc(file_path)
            # Save mfcc_feature as file
            output_file_path = os.path.join(mfcc_folder, file_name.replace('.wav', '.npy'))
            np.save(output_file_path, mfcc_feature)
            #print(f"MFCC features extracted and saved for {file_name} , shape = {mfcc_feature.shape}")

In [ ]:
extract_features_for_all_files(preprocessed_recordings_folder, mfcc_features_folder)

In [ ]:
mfcc_files= [f for f in os.listdir(mfcc_features_folder) if f.endswith('.npy')]
mfcc_features = {file: np.load(os.path.join(mfcc_features_folder, file)) for file in mfcc_files}

### Heat Map:

In [ ]:
def plot_mfcc_heatmaps(mfccs, n_mfcc = N_MFCC):
    for file_name, mfcc in mfccs.items():
        plt.figure(figsize=(15, 5))
        plt.title(f'MFCC Heatmap for {file_name}')
        librosa.display.specshow(mfcc, x_axis='time')
        plt.yticks(range(0, n_mfcc))
        plt.ylabel('MFCC')
        plt.colorbar(format='%+2.0f dB')
        plt.tight_layout()
        plt.show()

In [ ]:
files_with_index_0 = [f for f in os.listdir(mfcc_features_folder) if f.endswith('_0.npy')]
mfcc_features_index_0 = {file: np.load(os.path.join(mfcc_features_folder, file)) for file in files_with_index_0}
plot_mfcc_heatmaps(mfcc_features_index_0)

In [ ]:
def save_plot_mfcc_heatmaps(mfccs, folder_name, n_mfcc = N_MFCC):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    for file_name, mfcc in mfccs.items():
        plt.figure(figsize=(15, 5))
        plt.title(f'MFCC Heatmap for {file_name}')
        librosa.display.specshow(mfcc, x_axis='time')
        plt.yticks(range(0, n_mfcc))
        plt.ylabel('MFCC')
        plt.colorbar(format='%+2.0f dB')
        plt.tight_layout()
        full_path = os.path.join(folder_name, file_name.replace('.npy', ''))
        plt.savefig(full_path)
        plt.close()

In [ ]:
#save_plot_mfcc_heatmaps(mfcc_features, heatmaps_folder)

# Step Two: Prepare Data

## Prepare Data

In [ ]:
def prepare_data(mfccs, num_repeated_recoding = NUM_REPEATED_RECORDING):
    train_features, test_features = [], []
    train_labels, test_labels = [], []

    split_index = num_repeated_recoding * TRAIN_PERCENT
    for file_name, mfcc in mfccs.items():
        digit_label, speaker_name, index_str =  file_name[:-4].split('_')
        index = int(index_str)
        if index < split_index:
            train_features.append(mfcc.T)
            train_labels.append(file_name)
        else:
            test_features.append(mfcc.T)
            test_labels.append(file_name)

    return train_features, train_labels, test_features, test_labels

In [ ]:
train_features, train_labels, test_features, test_labels = prepare_data(mfcc_features, NUM_REPEATED_RECORDING)

In [ ]:
len(train_labels)

In [ ]:
len(test_labels)

# Step Three: Implementation 

In [ ]:
def print_detailed_report(arr1, arr2):
    t = 0
    f = 0
    for i in range(len(arr1)):
        if arr1[i] != arr2[i]:
            f = f + 1
        else:
            t = t + 1
    print(f"num_true = {t}, num_false = {f}")
    for i in range(len(arr1)):
        print(i)
        if arr1[i] != arr2[i]:
            print(Fore.RED + f"\033[1m{arr1[i], arr2[i]}\033[0m"+ Style.RESET_ALL)
        else:
            print(Fore.BLUE  + f"{arr1[i], arr2[i]}"+ Style.RESET_ALL)

In [ ]:
def extract_digit(label):
    digit_label = label.split('_')[0]
    return digit_label

def extract_speaker(label):
    speaker_label = label.split('_')[1]
    return speaker_label

Segmentation:

In [ ]:
# Organize training data by digit
training_data_by_digit = defaultdict(list)
for mfcc, label in zip(train_features, train_labels):
    digit = extract_digit(label)
    training_data_by_digit[digit].append(mfcc)

# Organize training data by speaker
training_data_by_speaker = defaultdict(list)
for mfcc, label in zip(train_features, train_labels):
    speaker = extract_speaker(label)
    training_data_by_speaker[speaker].append(mfcc)

In [ ]:
test_true_digits = [extract_digit(label) for label in test_labels]
test_true_speakers = [extract_speaker(label) for label in test_labels]

## Part 1:

#### Digit:

In [ ]:
hmm_models_digit_1 = {}

for digit, data in training_data_by_digit.items():
    lengths = [len(sequence) for sequence in data]
    X = np.concatenate(data)
    model = hmm.GaussianHMM(n_components = NUM_STATE)
    model.fit(X, lengths)
    hmm_models_digit_1[digit] = model


In [ ]:
test_predictions_digits_1 = []

for mfcc in test_features:
    best_score, best_digit = float("-inf"), None
    for digit, model in hmm_models_digit_1.items():
        score = model.score(mfcc)
        if score > best_score:
            best_score, best_digit = score, digit
    test_predictions_digits_1.append(best_digit)


In [ ]:
print_detailed_report(test_true_digits, test_predictions_digits_1)

#### Speaker:

In [ ]:
hmm_models_speaker_1 = {}

for speaker, data in training_data_by_speaker.items():
    lengths = [len(sequence) for sequence in data]
    X = np.concatenate(data)
    model = hmm.GaussianHMM(n_components = NUM_STATE)
    model.fit(X, lengths)
    hmm_models_speaker_1[speaker] = model


In [ ]:
test_predictions_speakers_1 = []

for mfcc in test_features:
    best_score, best_speaker = float("-inf"), None
    for speaker, model in hmm_models_speaker_1.items():
        score = model.score(mfcc)
        if score > best_score:
            best_score, best_speaker = score, speaker
    test_predictions_speakers_1.append(best_speaker)


In [ ]:
print_detailed_report(test_true_speakers, test_predictions_speakers_1)

## Part 2:

### Define HMM class

In [ ]:
class HMM:
    def __init__(self, num_hidden_states):
        self.num_hidden_states = num_hidden_states
        self.rand_state = np.random.RandomState(1)

        self.initial_prob = self._normalize(self.rand_state.rand(self.num_hidden_states, 1))
        self.transition_matrix = self._stochasticize(self.rand_state.rand(self.num_hidden_states, self.num_hidden_states))

        self.mean = None
        self.covariances = None
        self.num_dimensions = None

    def _forward(self, observation_matrix):
        log_likelihood = 0.
        T = observation_matrix.shape[1]
        alpha = np.zeros(observation_matrix.shape)

        for t in range(T):
            if t == 0:
                alpha[:, t] = observation_matrix[:, t] * self.initial_prob.flatten() 
                ## TODO: Forward algorithm for the first time step
            else:
                alpha[:, t] = observation_matrix[:, t] * np.dot(alpha[:, t-1], self.transition_matrix)
                ## TODO: Forward algorithm for the next time steps

            alpha_sum = np.sum(alpha[:, t])
            alpha[:, t] /= alpha_sum
            log_likelihood += np.log(alpha_sum)

        return log_likelihood, alpha

    def _backward(self, observation_matrix):
        T = observation_matrix.shape[1]
        beta = np.zeros(observation_matrix.shape)

        beta[:, -1] = np.ones(observation_matrix.shape[0])

        for t in range(T - 1)[::-1]:
            beta[:, t] = np.dot(self.transition_matrix.T,(observation_matrix[:, t+1] * beta[:, t+1]))
            ## TODO: Backward algorithm for the time steps of the HMM
            beta[:, t] /= np.sum(beta[:, t])

        return beta

    def _state_likelihood(self, obs):
        obs = np.atleast_2d(obs)
        B = np.zeros((self.num_hidden_states, obs.shape[1]))

        for s in range(self.num_hidden_states):
            mean_T =self.mean[:, s].T
            covariance_T = self.covariances[:, :, s].T
            obs_T = obs.T
            B[s, :] = scipy.stats.multivariate_normal.pdf(x = obs_T, mean=mean_T, cov=covariance_T)
            ## TODO: Compute the likelihood of observations with multivariate normal pdf
        return B

    def _normalize(self, x):
        return (x + (x == 0)) / np.sum(x)

    def _stochasticize(self, x):
        return (x + (x == 0)) / np.sum(x, axis=0)

    def _em_init(self, obs):
        if self.num_dimensions is None:
            self.num_dimensions = obs.shape[0]
        if self.mean is None:
            subset = self.rand_state.choice(
                np.arange(self.num_dimensions), size=self.num_hidden_states, replace=False)
            self.mean = obs[:, subset]
        if self.covariances is None:
            self.covariances = np.zeros(
                (self.num_dimensions, self.num_dimensions, self.num_hidden_states))
            self.covariances += np.diag(np.diag(np.cov(obs)))[:, :, None]

        return self

    def _em_step(self, obs):
        obs = np.atleast_2d(obs)
        T = obs.shape[1]

        B = self._state_likelihood(obs) ## TODO

        log_likelihood, alpha = self._forward(B) ## TODO
        beta = self._backward(B) ## TODO

        xi_sum = np.zeros((self.num_hidden_states, self.num_hidden_states))
        gamma = np.zeros((self.num_hidden_states, T))

        for t in range(T - 1):
            partial_sum = self.transition_matrix.T * np.inner(np.outer(B[:, t + 1],beta[:, t + 1]), alpha[:, t]) ## TODO
            xi_sum += self._normalize(partial_sum)
            partial_g = alpha[:, t] * beta[:, t] ## TODO
            gamma[:, t] = self._normalize(partial_g)
        partial_g = alpha[:, -1] * beta[:, -1] ## TODO
        gamma[:, -1] = self._normalize(partial_g)

        expected_prior = gamma[:, 0] ## TODO
        expected_transition = self._stochasticize(xi_sum)

        expected_covariances = np.zeros(
            (self.num_dimensions, self.num_dimensions, self.num_hidden_states))
        expected_covariances += .01 * np.eye(self.num_dimensions)[:, :, None]

        gamma_state_sum = np.sum(gamma, axis=1)
        gamma_state_sum = gamma_state_sum + (gamma_state_sum == 0)

        expected_mean = np.zeros((self.num_dimensions, self.num_hidden_states))
        for s in range(self.num_hidden_states):
            gamma_obs = obs * gamma[s, :]
            expected_mean[:, s] = np.sum(
                gamma_obs, axis=1) / gamma_state_sum[s]

        self.initial_prob = expected_prior
        self.mean = expected_mean
        self.transition_matrix = expected_transition

        return log_likelihood

    def train(self, obs, num_iterations=1):
        for i in range(num_iterations):


            self._em_init(obs)
            self._em_step(obs)
        return self

    def score(self, obs):
        B = self._state_likelihood(obs)
        log_likelihood, _ = self._forward(B)
        return log_likelihood

### Digit:

In [ ]:
hmm_models_digit_2 = {}

for digit, data in training_data_by_digit.items():
    model = HMM(num_hidden_states = NUM_STATE)
    X = np.concatenate(data)
    model.train(X.T, NUM_ITERATION)
    hmm_models_digit_2[digit] = model

In [ ]:
test_predictions_digits_2 = []

for mfcc in test_features:
    best_score, best_digit = float("-inf"), None
    for digit, model in hmm_models_digit_2.items():
        score = model.score(mfcc.T)
        if score > best_score:
            best_score, best_digit = score, digit
    test_predictions_digits_2.append(best_digit)

In [ ]:
print_detailed_report(test_true_digits, test_predictions_digits_2)

### Speaker:

In [ ]:
hmm_models_speaker_2 = {}

for speaker, data in training_data_by_speaker.items():
    model = HMM(num_hidden_states = NUM_STATE)
    X = np.concatenate(data)
    model.train(X.T, NUM_ITERATION)
    hmm_models_speaker_2[speaker] = model

In [ ]:
test_predictions_speakers_2 = []

for mfcc in test_features:
    best_score, best_speker = float("-inf"), None
    for speaker, model in hmm_models_speaker_2.items():
        score = model.score(mfcc.T)
        if score > best_score:
            best_score, best_speker = score, speaker
    test_predictions_speakers_2.append(best_speker)

In [ ]:
print_detailed_report(test_true_speakers, test_predictions_speakers_2)

# Step Four: Evaluation and Analysis

## Confusion Matrix:

In [ ]:
def create_confusion_matrix(y_true, y_pred):
    classes = sorted(set(y_true))
    class_indices = {cls: i for i, cls in enumerate(classes)}
    matrix = [[0 for _ in classes] for _ in classes]
    for actual, predicted in zip(y_true, y_pred):
        i = class_indices[actual]
        j = class_indices[predicted]
        matrix[i][j] += 1

    return matrix

In [ ]:
def plot_confusion_matrix(matrix, classes):
    fig, ax = plt.subplots()
    cax = ax.matshow(matrix, cmap=plt.cm.Spectral)
    fig.colorbar(cax)

    plt.xticks(np.arange(len(classes)), classes, rotation=45)
    plt.yticks(np.arange(len(classes)), classes)

    ax.set_xlabel('Predicted')
    ax.xaxis.set_label_position('top')
    ax.set_ylabel('True')

    for (i, j), val in np.ndenumerate(matrix):
        ax.text(j, i, f'{val}', ha='center', va='center', color='black')

    plt.show()

In [ ]:
def create_plot_confusion_matrix(true_vals, predict_vals, vals, type, part):
    print(f"for {part}:")
    cm = create_confusion_matrix(true_vals, predict_vals)
    plot_confusion_matrix(cm, vals)
    return np.array(cm)

## Calculate Metrics:

In [ ]:
def calculate_metrics(confusion_matrix, class_index):
    TP = confusion_matrix[class_index, class_index]
    TN = np.sum(confusion_matrix) - np.sum(confusion_matrix[class_index, :]) - np.sum(confusion_matrix[:, class_index]) + TP
    FP = np.sum(confusion_matrix[:, class_index]) - TP
    FN = np.sum(confusion_matrix[class_index, :]) - TP
    return TP, TN, FP, FN
        

## Accuracy:

In [ ]:
def print_accuracy(confusion_matrix, vals):
    accs = []
    for i in range(len(vals)):
        TP, TN, FP, FN = calculate_metrics(confusion_matrix, i)
        ALL = TP + TN + FP + FN
        T = TP + TN
        acc = T/ALL
        accs.append(acc)
        print(f"  {vals[i]} : {acc* 100:.2f}")
    print(f"macro avg accuracy = {sum(accs)/len(accs) * 100:.2f}%")
    print("--" * 15)

In [ ]:
def print_accuracies(cm_1, cm_2, vals, type):
    print("accuracies for " + type + "-" + "part 1:")
    print_accuracy(cm_1, vals)
    print("accuracies for " + type + "-" + "part 2:")
    print_accuracy(cm_2, vals)

## Precision:

In [ ]:
def print_precision(confusion_matrix, vals):
    precs = []
    for i in range(len(vals)):
        TP, TN, FP, FN = calculate_metrics(confusion_matrix, i)
        precision = TP / (TP + FP)
        precs.append(precision)
        print(f"  {vals[i]} : {precision* 100:.2f}")
    print(f"macro avg precision = {sum(precs)/len(precs) * 100:.2f}%")
    print("--" * 15)

In [ ]:
def print_precisions(cm_1, cm_2, vals, type):
    print("precision for " + type + "-" + "part 1:")
    print_precision(cm_1, vals)
    print("precision for " + type + "-" + "part 2:")
    print_precision(cm_2, vals)

## Recall

In [ ]:
def print_recall(confusion_matrix, vals):
    recs = []
    for i in range(len(vals)):
        TP, TN, FP, FN = calculate_metrics(confusion_matrix, i)
        recall = TP / (TP + FN)
        recs.append(recall)
        print(f"  {vals[i]} : {recall* 100:.2f}")
    print(f"macro avg recall = {sum(recs)/len(recs) * 100:.2f}%")
    print("--" * 15)

In [ ]:
def print_recalls(cm_1, cm_2, vals, type):
    print("recall for " + type + "-" + "part 1:")
    print_recall(cm_1, vals)
    print("recall for " + type + "-" + "part 2:")
    print_recall(cm_2, vals)

## F1 Score:

In [ ]:
def print_f1(confusion_matrix, vals):
    f1s = []
    for i in range(len(vals)):
        TP, TN, FP, FN = calculate_metrics(confusion_matrix, i)
        p =  TP / (TP + FP)
        r = TP / (TP + FN)
        f1 = 2 * r * p / (r + p)
        f1s.append(f1)
        print(f"  {vals[i]} : {f1* 100:.2f}")
    print(f"macro avg F1 score = {sum(f1s)/len(f1s) * 100:.2f}%")
    print("--" * 15)

In [ ]:
def print_f1_scores(cm_1, cm_2, vals, type):
    print("F1 score for " + type + "-" + "part 1:")
    print_f1(cm_1, vals)
    print("F1 score for " + type + "-" + "part 2:")
    print_f1(cm_2, vals)

### Visualization:

In [ ]:
def all_metrics(confusion_matrix, vals):
    f1s = []
    recs = []
    accs = []
    precs = []
    for i in range(len(vals)):
        TP, TN, FP, FN = calculate_metrics(confusion_matrix, i)
        ALL = TP + TN + FP + FN
        T = TP + TN 
        acc = T/ALL
        pre =  TP / (TP + FP)
        recall = TP / (TP + FN)
        f1 = 2 * recall * pre / (recall + pre)
        f1s.append(f1 * 100)
        recs.append(recall* 100)
        accs.append(acc * 100)
        precs.append(pre * 100)
    return  accs, precs, recs, f1s

In [ ]:
def all_metric_2_approch(cm_1, cm_2, vals, type):
    accs1, precs1, recs1, f1s1 = all_metrics(cm_1, vals)
    accs2, precs2, recs2, f1s2 = all_metrics(cm_2, vals)
    
    x = np.arange(len(vals))
    width = 0.35 
    fig, axs = plt.subplots(2, 2, figsize=(12, 10)) 

    # Plotting accuracy
    rects1 = axs[0, 0].bar(x - width/2, accs1, width, label='Accuracy - part 1')
    rects2 = axs[0, 0].bar(x + width/2, accs2, width, label='Accuracy - part 2')
    axs[0, 0].set_title('Accuracy Comparison')
    axs[0, 0].set_xticks(x)
    axs[0, 0].set_xticklabels(vals)
    axs[0, 0].legend()

    avg_acc1 = np.mean(accs1)
    avg_acc2 = np.mean(accs2)
    var_acc1 = np.var(accs1)
    var_acc2 = np.var(accs2)
    axs[0, 0].annotate(f'Avg 1: {avg_acc1:.2f}, Var 1: {var_acc1:.2f}\nAvg 2: {avg_acc2:.2f}, Var 2: {var_acc2:.2f}', 
                       xy=(0.5, 0.95), xycoords='axes fraction', ha='center', va='center')

    # Plotting precision
    rects1 = axs[0, 1].bar(x - width/2, precs1, width, label='Precision - part 1')
    rects2 = axs[0, 1].bar(x + width/2, precs2, width, label='Precision - part 2')
    axs[0, 1].set_title('Precision Comparison')
    axs[0, 1].set_xticks(x)
    axs[0, 1].set_xticklabels(vals)
    axs[0, 1].legend()

    avg_prec1 = np.mean(precs1)
    avg_prec2 = np.mean(precs2)
    var_prec1 = np.var(precs1)
    var_prec2 = np.var(precs2)
    axs[0, 1].annotate(f'Avg 1: {avg_prec1:.2f}, Var 1: {var_prec1:.2f}\nAvg 2: {avg_prec2:.2f}, Var 2: {var_prec2:.2f}', 
                       xy=(0.5, 0.95), xycoords='axes fraction', ha='center', va='center')

    # Plotting recall
    rects1 = axs[1, 0].bar(x - width/2, recs1, width, label='Recall - part 1')
    rects2 = axs[1, 0].bar(x + width/2, recs2, width, label='Recall - part 2')
    axs[1, 0].set_title('Recall Comparison')
    axs[1, 0].set_xticks(x)
    axs[1, 0].set_xticklabels(vals)
    axs[1, 0].legend()

    avg_recall1 = np.mean(recs1)
    avg_recall2 = np.mean(recs2)
    var_recall1 = np.var(recs1)
    var_recall2 = np.var(recs2)
    axs[1, 0].annotate(f'Avg 1: {avg_recall1:.2f}, Var 1: {var_recall1:.2f}\nAvg 2: {avg_recall2:.2f}, Var 2: {var_recall2:.2f}', 
                       xy=(0.5, 0.95), xycoords='axes fraction', ha='center', va='center')

    # Plotting F1 score
    rects1 = axs[1, 1].bar(x - width/2, f1s1, width, label='F1 Score - part 1')
    rects2 = axs[1, 1].bar(x + width/2, f1s2, width, label='F1 Score - part 2')
    axs[1, 1].set_title('F1 Score Comparison')
    axs[1, 1].set_xticks(x)
    axs[1, 1].set_xticklabels(vals)
    axs[1, 1].legend()

    avg_f1_1 = np.mean(f1s1)
    avg_f1_2 = np.mean(f1s2)
    var_f1_1 = np.var(f1s1)
    var_f1_2 = np.var(f1s2)
    axs[1, 1].annotate(f'Avg 1: {avg_f1_1:.2f}, Var 1: {var_f1_1:.2f}\nAvg 2: {avg_f1_2:.2f}, Var 2: {var_f1_2:.2f}', 
                       xy=(0.5, 0.95), xycoords='axes fraction', ha='center', va='center')

    plt.xticks(rotation=45, ha='right') 
    plt.tight_layout() 
    
    plt.show()

## Calculations: 

### Digit:

In [ ]:
DIGITS = list(set(test_true_digits))
DIGITS = sorted(DIGITS)
print(DIGITS)
cm_d_1 = create_plot_confusion_matrix(test_true_digits,test_predictions_digits_1, DIGITS, "digits", "part 1")
cm_d_2 = create_plot_confusion_matrix(test_true_digits,test_predictions_digits_2, DIGITS, "digits", "part 2")

In [ ]:
print_accuracies(cm_d_1, cm_d_2, DIGITS, "digits")

In [ ]:
print_precisions(cm_d_1, cm_d_2, DIGITS, "digits")

In [ ]:
print_recalls(cm_d_1, cm_d_2, DIGITS, "digits")

In [ ]:
print_f1_scores(cm_d_1, cm_d_2, DIGITS, "digits")

In [ ]:
all_metric_2_approch(cm_d_1, cm_d_2, DIGITS, "digits")

### Speaker:

In [ ]:
SPEAKERS = list(set(test_true_speakers))
SPEAKERS = sorted(SPEAKERS)
print(SPEAKERS)
cm_s_1 = create_plot_confusion_matrix(test_true_speakers,test_predictions_speakers_1, SPEAKERS, "speakers", "part 1")
cm_s_2 = create_plot_confusion_matrix(test_true_speakers,test_predictions_speakers_2, SPEAKERS, "speakers", "part 2")

In [ ]:
print_accuracies(cm_s_1, cm_s_2, SPEAKERS, "speakers")

In [ ]:
print_precisions(cm_s_1, cm_s_2, SPEAKERS, "speakers")

In [ ]:
print_recalls(cm_s_1, cm_s_2, SPEAKERS, "speakers")

In [ ]:
print_f1_scores(cm_s_1, cm_s_2, SPEAKERS, "speakers")

In [ ]:
all_metric_2_approch(cm_s_1, cm_s_2, SPEAKERS, "speakers")